## Betfair

In [137]:
import os
import pandas as pd
import datetime as dt
from betfairlightweight import APIClient, filters as f
uname = os.environ['betfair_username']
pword = os.environ['betfair_password']
key = os.environ['betfair_key']
bf = APIClient(uname, pword, app_key=key,
               certs='/Users/jonaspaulwestermann/Dev/marchmadness/certs',
               locale='spain', lightweight=True)
bf.login()

{'loginStatus': 'SUCCESS',
 'sessionToken': 'YfpgGOhWycLTwrf/bMiyKl4zKCO9w4n8R3Lz+QPN/vw='}

### Competition

In [138]:
ncaa_competition = next(e for e in bf.betting.list_competitions() if e['competition']['name'] == 'NCAAB')
ncaa_competition

{'competition': {'id': '10861937', 'name': 'NCAAB'},
 'competitionRegion': 'International',
 'marketCount': 10}

### Events

In [143]:
ncaa_filter = f.market_filter(competition_ids=[ncaa_competition['competition']['id']])
ncaa_games = pd.DataFrame([g['event'] for g in bf.betting.list_events(filter=ncaa_filter)]).set_index('id')
ncaa_games['time_to_open'] = (pd.to_datetime(ncaa_games['openDate']) - dt.datetime.now())
print(ncaa_games.shape)
ncaa_games.sort_values('time_to_open').head(3)

(10, 4)


,name,openDate,timezone,time_to_open
id,,,,
28620195,Georgia Tech @ Boston College,2018-03-06T17:00:00.000Z,GMT,11:51:35.059862
28620187,Pittsburgh @ Notre Dame,2018-03-06T19:30:00.000Z,GMT,14:21:35.059862
28620192,Northern Arizona @ No. Colorado,2018-03-06T22:30:00.000Z,GMT,17:21:35.059862


### Market Types

In [144]:
ncaa_filter = f.market_filter(competition_ids=[ncaa_competition['competition']['id']])
ncaa_market_types = pd.DataFrame(bf.betting.list_market_types(filter=ncaa_filter)).set_index('marketType')
print(ncaa_market_types.shape)
ncaa_market_types

(1, 1)


,marketCount
marketType,
MATCH_ODDS,10


### Market Catalogue

In [145]:
ncaa_filter = f.market_filter(competition_ids=[ncaa_competition['competition']['id']])
max_results = int(ncaa_market_types.loc['MATCH_ODDS','marketCount'])
ncaa_market_catalogue = bf.betting.list_market_catalogue(filter=ncaa_filter, max_results=max_results)
ncaa_market_catalogue = pd.DataFrame(ncaa_market_catalogue).set_index('marketId')
print(ncaa_market_catalogue.shape)
ncaa_market_catalogue.sample(3)

(10, 2)


,marketName,totalMatched
marketId,,
1.140978035,Moneyline,0.0
1.140978326,Moneyline,0.0
1.140978762,Moneyline,0.0


### Market Book

In [146]:
ncaa_market_book = bf.betting.list_market_book(market_ids=ncaa_market_catalogue.index)
ncaa_market_book = pd.DataFrame(ncaa_market_book)
print(ncaa_market_book.shape)
ncaa_market_id = ncaa_market_book.sample().marketId
ncaa_market_book.sample(3)

(10, 16)


,betDelay,bspReconciled,complete,crossMatching,inplay,isMarketDataDelayed,marketId,numberOfActiveRunners,numberOfRunners,numberOfWinners,runners,runnersVoidable,status,totalAvailable,totalMatched,version
0,0,False,True,False,False,True,1.140977744,2,2,1,"[{'selectionId': 352102, 'handicap': 0.0, 'sta...",False,OPEN,0.0,0.0,2095825034
5,0,False,True,False,False,True,1.140978472,2,2,1,"[{'selectionId': 5145595, 'handicap': 0.0, 'st...",False,OPEN,0.0,0.0,2095829565
4,0,False,True,False,False,True,1.140978326,2,2,1,"[{'selectionId': 731888, 'handicap': 0.0, 'sta...",False,OPEN,0.0,0.0,2095829069


In [156]:
ncaa_market_profit_and_loss = bf.betting.list_market_profit_and_loss(market_ids=ncaa_market_catalogue.index,
                                                                     include_settled_bets=True)
ncaa_market_profit_and_loss = pd.DataFrame(ncaa_market_profit_and_loss)
print(ncaa_market_profit_and_loss.shape)
ncaa_market_id = ncaa_market_profit_and_loss.sample().marketId
ncaa_market_profit_and_loss.sample(3)

(10, 2)


,marketId,profitAndLosses
2,1.140978035,"[{'selectionId': 284246, 'ifWin': 0.0}, {'sele..."
6,1.140978617,"[{'selectionId': 5145582, 'ifWin': 0.0}, {'sel..."
4,1.140978326,"[{'selectionId': 731888, 'ifWin': 0.0}, {'sele..."


In [160]:
ncaa_market_profit_and_loss.sample().squeeze().profitAndLosses

[{'ifWin': 0.0, 'selectionId': 728491}, {'ifWin': 0.0, 'selectionId': 723288}]